In [27]:
import os 
import chromadb 

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

In [16]:
def read_pdf(dir_path):
    file_loader = PyPDFDirectoryLoader(dir_path)
    
    return file_loader.load()

In [17]:
docs = read_pdf("./data/")

In [18]:
len(docs)

3

In [19]:
for doc in docs: 
    print(doc)
    print()
    print(100*"#")
    print()

page_content='A Habilidade Mais Importante na Era da Inteligência Artificial 
 
A pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo, já que 
tudo, desde reuniões até consultas médicas, ficou online. Isso pode soar como algo super 
positivo.  
Para dezenas de milhões de trabalhadores, não. 
Eles talvez não tenham as habilidades necessárias para competir nesse novo mundo. Eles são os 
contadores, os digitadores, os secretários executivos, procurando trabalho em uma nova 
economia na qual as pessoas contratadas têm títulos como “Engenheiro de Nuv em” ou “Hacker 
de Crescimento” em seus currículos. Sem um esforço concentrado para retreiná-los, descobriram 
os pesquisadores da RAND Europe, eles provavelmente serão deixados para trás. 
E não apenas eles. O custo dessa crescente lacuna de habilidades será medido em trilhões de 
dólares e recairá mais fortemente em lugares que não possuem infraestrutura digital confiável, 
como acesso à Internet ou fluência gener

In [20]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [21]:
gemini_embeddings.embed_query("What is quantum mechanics?")

[0.009460452,
 -0.08125154,
 0.018852543,
 -0.06393108,
 0.048903946,
 -0.01366113,
 0.00755695,
 -0.02353137,
 -0.00040936918,
 0.03546514,
 -0.022752736,
 0.04709375,
 -0.008799185,
 -0.04030077,
 -0.00895462,
 0.03876839,
 0.049946036,
 0.013551037,
 0.03956643,
 -0.02740316,
 0.015613765,
 -0.0022608102,
 0.02165617,
 0.010558679,
 0.015244326,
 -0.041065194,
 0.017983308,
 -0.066469595,
 0.020358281,
 0.038997494,
 -0.078209415,
 0.018321985,
 -0.023352321,
 0.010387016,
 0.0023501494,
 -0.03475852,
 0.055649478,
 -0.011547425,
 -0.014292862,
 0.015659992,
 -0.0031742807,
 0.016056785,
 -0.022485215,
 -0.036626216,
 0.004542658,
 -0.048943892,
 -0.011121343,
 0.02006516,
 0.004634597,
 -0.09492328,
 0.04230494,
 -0.00051806285,
 0.022934156,
 -0.034149997,
 0.02666348,
 -0.024049928,
 0.013470102,
 -0.041393455,
 -0.0070100557,
 -0.040159684,
 0.010674196,
 -0.011180459,
 -0.040073697,
 0.031408973,
 0.010154207,
 0.0015024847,
 -0.005304193,
 0.06929477,
 0.026252186,
 -0.0371082

In [23]:
index_name = "intex-test"

index = Chroma.from_documents(
    documents=docs, 
    embedding=gemini_embeddings, 
    collection_name=index_name
)

In [24]:
def similarity_search(query, vector_index, k=2):

    return vector_index.similarity_search(query, k)

In [34]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.3,
    google_api_key=os.environ.get("GEMINI_API_KEY")
)

In [35]:
template = """
Responda a pergunta com base apenas no contexto abaixo:
{context}

Pergunta: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [36]:
chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [37]:
def get_response(query):

    docs = index.similarity_search(query) # Retrieve documents
    
    # Generate context
    context = "\n\n".join([doc.page_content for doc in docs])
    
    response = chain.invoke({
        "context": context,
        "question": query
    })
    
    return response

In [39]:
response1 = get_response("O que a pesquisa recente do SalesForce descobriu?")

In [40]:
from pprint import pprint

pprint(response1)

('Uma pesquisa recente da Salesforce descobriu que três quartos dos '
 'trabalhadores do mundo se sentem despreparados para os empregos que podem '
 'encontrar do outro lado do marco em que o número de horas trabalhadas pelas '
 'máquinas será igual ao número de horas trabalhadas pelos humanos.')
